In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import urllib

# ================================================================================
# Función para convertir la URL de Google Sheet en CSV
# ================================================================================
def get_google_sheet_csv_url(sheet_url: str) -> str:
    # Extrae el ID del documento desde la URL
    file_id = sheet_url.split("/d/")[1].split("/")[0]
    # Genera la URL directa para descargar como CSV
    return f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv"

# ================================================================================
# Función para cargar y transformar los datos
# ================================================================================
def ingest_and_clean_google_sheet(sheet_url: str) -> pd.DataFrame:
    csv_url = get_google_sheet_csv_url(sheet_url)  # Convierte URL a CSV
    df = pd.read_csv(csv_url)                      # Carga los datos como DataFrame
    
    # Limpieza básica: nombres de columnas en minúsculas y sin espacios
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    
    # Agrega una columna con la fecha de ingesta
    df["fecha_ingesta"] = datetime.today().strftime('%Y-%m-%d')
    
    return df

# ================================================================================
# Función para guardar el DataFrame a un archivo CSV local
# ================================================================================
def save_data(df: pd.DataFrame, output_path: str):
    df.to_csv(output_path, index=False)

# ================================================================================
# Parámetros de ejecución
# ================================================================================

# Pegá acá tu URL pública de Google Sheet
sheet_url = "https://docs.google.com/spreadsheets/d/1AbCdEfG1234567xyz/edit#gid=0"

# Ejecuta el flujo completo
df_final = ingest_and_clean_google_sheet(sheet_url)
save_data(df_final, "./data/salida_google_sheet.csv")  # Cambiá la ruta si querés


# ================================================================================
# Función para cargar en PostgreSQL (Azure)
# ================================================================================
def load_to_postgresql(df: pd.DataFrame, table_name: str, user: str, password: str, host: str, port: str, db: str):
    # Codifica la contraseña si tiene caracteres especiales
    password = urllib.parse.quote_plus(password)

    # Crea la URL de conexión para SQLAlchemy
    engine_url = f"postgresql://{user}:{password}@{host}:{port}/{db}"
    
    # Crear engine SQLAlchemy
    engine = create_engine(engine_url)

    # Carga en la tabla (append = agrega, replace = reemplaza)
    df.to_sql(table_name, engine, if_exists="replace", index=False)
    print(f"✅ Datos cargados exitosamente en la tabla: {table_name}")

# ================================================================================
# PARÁMETROS
# ================================================================================

# URL de tu Google Sheet (asegurate de que esté compartida públicamente o como "Cualquiera con el enlace")
sheet_url = "https://docs.google.com/spreadsheets/d/1AbCdEfG1234567xyz/edit#gid=0"

# Datos de tu PostgreSQL en Azure
POSTGRES_USER = "tu_usuario"
POSTGRES_PASSWORD = "tu_contraseña"
POSTGRES_HOST = "tu_host.postgres.database.azure.com"
POSTGRES_PORT = "5432"
POSTGRES_DB = "nombre_de_tu_base"
TABLE_NAME = "nombre_tabla_destino"

# ================================================================================
# FLUJO DE EJECUCIÓN
# ================================================================================
df_final = ingest_and_clean_google_sheet(sheet_url)
load_to_postgresql(
    df=df_final,
    table_name=TABLE_NAME,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    db=POSTGRES_DB
)
